In [4]:
import torch
import numpy as np
from torch.optim import Adam

import os
import json
import tqdm
from datetime import datetime

from torch.distributions import Normal, Poisson, MultivariateNormal
from gpytorch.kernels import RBFKernel, ScaleKernel
from torch import nn

from torch.utils.tensorboard import SummaryWriter
from bdp.models.utils.basic_setups import create_dir_and_writer

from bdp.models.gaussian_processes.gaussian_processes import multivariate_normal, white_noise_kernel

from bdp.models.random_fields.blocks_utils_torch import calculate_determinant_and_inverse_covariance_history_torch
from bdp.models.random_fields.blocks_utils_torch import obtain_location_index_to_realization, obtain_blocks
from bdp.models.random_fields.blocks_utils_torch import log_probability_from_blocks

In [5]:
locations_dimension = 2
prior_locations_mean =  0.
prior_locations_std = 6.69
number_of_realizations = 100
birth_intensity = 2.
kernel_sigma = 10.
kernel_lenght_scales = [1., 2.]

locations_prior = Normal(torch.full((locations_dimension,), prior_locations_mean),
                         torch.full((locations_dimension,), prior_locations_std))

# Births
birth_distribution = Poisson(birth_intensity)
birth_numbers = birth_distribution.sample((number_of_realizations,)).long()
assets_in_the_market = birth_numbers.cumsum(dim=0)
total_assets_in_history = assets_in_the_market[-1]

# Locations
locations_history = locations_prior.sample(sample_shape=(total_assets_in_history,))
    
def define_kernel(kernel_sigma, kernel_lenght_scales):
    kernel = ScaleKernel(RBFKernel(ard_num_dims=locations_dimension, requires_grad=True),
                         requires_grad=True) + white_noise_kernel()

    kernel_hypers = {"raw_outputscale": torch.tensor(kernel_sigma),
                     "base_kernel.raw_lengthscale": torch.tensor(kernel_lenght_scales)}

    kernel.kernels[0].initialize(**kernel_hypers)
    kernel_eval = lambda locations: kernel(locations, locations).evaluate().type(torch.float64)
    
    return kernel, kernel_eval
    
def define_covariance_matrix_from_kernel(number_of_realizations,birth_intensity,kernel_sigma,kernel_lenght_scales):    
    number_of_realizations = number_of_realizations

    # Kernel
    kernel, kernel_eval = define_kernel(kernel_sigma,kernel_lenght_scales)
    covariance_diffusion_history = kernel_eval(locations_history)
    
    return covariance_diffusion_history

In [7]:
mu = torch.ones(total_assets_in_history).type(torch.float64)*0.01
covariance_matrix = define_covariance_matrix_from_kernel(number_of_realizations,birth_intensity,kernel_sigma,kernel_lenght_scales)
# generate data
distribution = MultivariateNormal(mu,covariance_matrix)
sample = distribution.sample(sample_shape=(number_of_realizations,))
for i,current_number_of_assets in enumerate(assets_in_the_market):
    sample[i,current_number_of_assets:] = torch.zeros_like(sample[i,current_number_of_assets:])
log_returns = sample

In [8]:
determinants_history, inverse_covariance_history = calculate_determinant_and_inverse_covariance_history_torch(
    assets_in_the_market, covariance_matrix, False)

In [9]:
print("Real {0} Estimated {1}".format(torch.det(covariance_matrix),determinants_history[-1]))
print("Last Element of Diagonal {0}".format(torch.matmul(covariance_matrix,inverse_covariance_history[-1])[-1,-1]))

Real 7.47916565598642e-23 Estimated 7.479653520082225e-23
Last Element of Diagonal 1.0000000132827986


In [10]:
class MertonBirthKernel(nn.Module):

    def __init__(self, **kwargs):
        nn.Module.__init__(self)
        self.locations_dimension = kwargs.get("locations_dimension")
        self.define_deep_parameters()
        
    def define_kernel(self):
        kernel = ScaleKernel(RBFKernel(ard_num_dims=self.locations_dimension, requires_grad=True),
                             requires_grad=True) + white_noise_kernel()

        kernel_hypers = {"raw_outputscale": torch.tensor(self.kernel_sigma),
                         "base_kernel.raw_lengthscale": torch.tensor(self.kernel_lenght_scales)}

        kernel.kernels[0].initialize(**kernel_hypers)
        kernel_eval = lambda locations: kernel(locations, locations).evaluate().type(torch.float64)

        return kernel, kernel_eval

    def forward(self, locations_history):
        kernel, kernel_eval = define_kernel(kernel_sigma,kernel_lenght_scales)
        covariance_diffusion_history = kernel_eval(locations_history)
        return covariance_diffusion_history

    def define_deep_parameters(self):
        self.kernel_sigma = nn.Parameter(torch.Tensor([1.]))
        self.kernel_lenght_scales = nn.Parameter(torch.Tensor([20.,30.]))

In [11]:
number_of_epochs = 100
mbk = MertonBirthKernel(locations_dimension=locations_dimension)
mbk.train()
optimizer = Adam(mbk.parameters(), lr=0.001)
for i in tqdm.tqdm(range(number_of_epochs)):
    covariance_matrix = mbk(locations_history)
    determinants_history, inverse_covariance_history = calculate_determinant_and_inverse_covariance_history_torch(assets_in_the_market, covariance_matrix, False)
    loss = log_probability_from_blocks(sample,mu,assets_in_the_market,determinants_history,inverse_covariance_history).sum()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  0%|          | 0/100 [00:00<?, ?it/s]


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.DoubleTensor []], which is output 0 of AsStridedBackward0, is at version 100; expected version 99 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).